In [1]:
import os
import cv2
import sys

import keras

import pickle
import random
import itertools
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from   PIL import Image
from   functools import partial

from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
from sklearn.utils import shuffle

from ssd import SSD300, MultiboxLoss, BBoxUtility

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [2]:
def passthrough(img):
    return img

class UdacityDataset():

    @staticmethod
    def grayscale(rgb):
        return rgb.dot([0.299, 0.587, 0.114])

    @staticmethod
    def saturation(rgb, saturation_var=0.5):
        gs = UdacityDataset.grayscale(rgb)
        alpha = 2 * np.random.random() * saturation_var 
        alpha += 1 - saturation_var
        rgb = rgb * alpha + (1 - alpha) * gs[:, :, None]
        return np.clip(rgb, 0, 255)

    @staticmethod
    def brightness(rgb, saturation_var=0.5):
        alpha = 2 * np.random.random() * brightness_var 
        alpha += 1 - saturation_var
        rgb   = rgb * alpha
        return np.clip(rgb, 0, 255)

    @staticmethod
    def contrast(rgb, contrast_var=0.5):
        gs = UdacityDataset.grayscale(rgb).mean() * np.ones_like(rgb)
        alpha = 2 * np.random.random() * contrast_var 
        alpha += 1 - contrast_var
        rgb = rgb * alpha + (1 - alpha) * gs
        return np.clip(rgb, 0, 255)

    @staticmethod
    def lighting(img, lighting_std=0.5):
        cov = np.cov(img.reshape(-1, 3) / 255.0, rowvar=False)
        eigval, eigvec = np.linalg.eigh(cov)
        noise = np.random.randn(3) * lighting_std
        noise = eigvec.dot(eigval * noise) * 255
        img += noise
        return np.clip(img, 0, 255)

    
    def cls2onehot(self, idx):
        res      = np.zeros((len(self.labels)+1,))
        res[idx] = 1
        return res

    def __init__(self, 
                 data_dir, 
                 config, 
                 new_size, 
                 batch_size=4, 
                 epochs = 2,
                 augmentation=[passthrough], 
                 priors_pkl='prior_boxes_ssd300.pkl'):
        super(UdacityDataset, self).__init__()
        self.data_dir      = data_dir
        self.config        = config
        self.new_size      = new_size
        self.batch_size    = batch_size
        self.epochs        = epochs
        self.augmentation  = list(set(augmentation)|set([passthrough]))
        self.labels        = ['car', 'truck', 'pedestrian', 'bicyclist',  'traffic light']
        self.priors        = pickle.load(open(priors_pkl, 'rb'))
        self.bbox_util     = BBoxUtility(len(self.labels)+2, self.priors)
        self.ids           = {
            'all': [
                x 
                for x in map(lambda x: x, os.listdir(self.data_dir)) 
                if 'jpg' in x
            ]
        }
        self.ids['train'] = self.ids['all'][:int(len(self.ids['all']) * 0.75)]
        self.ids['val']   = self.ids['all'][int(len(self.ids['all'])  * 0.75):]
        self.iterations   = {
            'all': len(self.ids['all'])//self.batch_size,
            'val': len(self.ids['val'])//self.batch_size,
            'train': len(self.ids['train'])//self.batch_size
        }

        self.n_classes    = len(self.labels)+2
        self.shape        = (new_size, new_size, 3)

        self.df       = pd.read_csv(
             os.path.join(self.data_dir, self.config)
        )

    def __len__(self):
        return len(self.ids['all'])

    def _get_image(self, index):
        return cv2.imread(
            os.path.join(self.data_dir, self.ids['all'][index])
        )
        return img

    def _get_annotation(self, index, hw):
        (h, w)   = hw
        boxes    = []
        labels   = []
        for idx, row in self.df[self.df['frame']==self.ids['all'][index]].iterrows():
            xmin = float(row['xmin']) / w
            ymin = float(row['ymin']) / h
            xmax = float(row['xmax']) / w
            ymax = float(row['ymax']) / h
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.cls2onehot(int(row['class_id'])))
        return np.hstack((boxes, labels))

    def generate(self, subset='all'):
        while True:
            for epoch in range(self.epochs):
                self.ids[subset] = shuffle(self.ids[subset])
                start            = -self.batch_size
                for _ in range(self.iterations[subset]):
                    boxes  = []
                    images = []
                    start += self.batch_size
                    for index in range(start, start + self.batch_size):
                        filename        = self.ids[subset][index]
                        augmentation    = np.random.choice(self.augmentation)
                        img             = self._get_image(index)
                        img = np.array(
                            cv2.resize(
                                img,
                                (self.new_size, self.new_size),
                                interpolation = cv2.INTER_AREA
                            ), 
                            dtype=np.float32
                        )
                        img              = augmentation(img)
                        #img              = (img / 127.0) - 1.0
                        annot_boxes      = self._get_annotation(index, (img.shape[0], img.shape[1]))
                        annot_boxes      = self.bbox_util.assign_boxes(annot_boxes)
                        boxes.append(annot_boxes)
                        images.append(img)
                    #print("images: ", np.array(images).shape)
                    #print("boxes:  ", np.array(boxes).shape)
                    yield preprocess_input(np.array(images)),  np.array(boxes)

class UdacityGenerator:
    def __new__(cls, 
                data_dir='./udacity_driving_datasets', 
                config='labels_trainval.csv', 
                new_size=300, 
                batch_size=4, 
                epochs = 2,
                augmentation=[passthrough]):
        udacity      = UdacityDataset(
            data_dir, config, new_size, batch_size, epochs, augmentation
        )
        info = {
            'labels':       udacity.labels,
            'length':       len(udacity),
            'batch_size':   batch_size
        }
        return udacity, info

In [3]:
udacity, info = UdacityGenerator()

In [4]:
model = SSD300(udacity.shape, num_classes=udacity.n_classes)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
freeze = ['input_1', 'conv1_1', 'conv1_2', 'pool1',
          'conv2_1', 'conv2_2', 'pool2',
          'conv3_1', 'conv3_2', 'conv3_3', 'pool3']

for L in model.layers:
    if L.name in freeze:
        L.trainable = False

In [6]:
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)

callbacks = [keras.callbacks.ModelCheckpoint('./checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                                             verbose=1,
                                             save_weights_only=True),
             keras.callbacks.LearningRateScheduler(schedule)]

In [7]:
base_lr = 3e-4
optim = keras.optimizers.Adam(lr=base_lr)
# optim = keras.optimizers.RMSprop(lr=base_lr)
# optim = keras.optimizers.SGD(lr=base_lr, momentum=0.9, decay=decay, nesterov=True)
model.compile(optimizer=optim,
              loss=MultiboxLoss(udacity.n_classes, neg_pos_ratio=2.0).compute_loss)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [8]:
history = model.fit_generator(udacity.generate(subset='train'), 
                              udacity.iterations['train'],
                              udacity.epochs, 
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=udacity.generate(subset='val'),
                              nb_val_samples=udacity.iterations['val'],
                              nb_worker=1)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/2
 100/4170 [..............................] - ETA: 7875s - loss: 9.2784

KeyboardInterrupt: 

In [8]:
images = []
img = cv2.imread('udacity_driving_datasets/1478020291199453460.jpg')
img = cv2.resize(img, (300, 300), interpolation = cv2.INTER_AREA)
images.append(img)
images = np.array(images, dtype=np.float32)
inputs = images.copy()
inputs = preprocess_input(inputs)

NameError: name 'cv2' is not defined

In [9]:
priors        = pickle.load(open('prior_boxes_ssd300.pkl', 'rb'))
bbox_util     = BBoxUtility(7, priors)
preds = model.predict(inputs, batch_size=1, verbose=1)
results = bbox_util.detection_out(preds)

NameError: name 'pickle' is not defined

In [10]:
for i, img in enumerate(images):
    # Parse the outputs.
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.35]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, 40)).tolist()

    plt.imshow(img / 255.)
    currentAxis = plt.gca()

    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))
        score = top_conf[i]
        label = int(top_label_indices[i])
        display_txt = '{:0.2f}, {}'.format(score, label)
        coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.show()